In [1]:
import google.generativeai as genai
import json
import re
import uuid
import time
import os
from typing import Dict, List, Tuple
from collections import deque
from datetime import datetime

/Users/toan/Environment/normCS/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class RateLimiter:
    def __init__(self, max_calls: int, time_frame: int):
        self.max_calls = max_calls
        self.time_frame = time_frame 
        self.calls = deque()
    
    def wait_if_needed(self, verbose: bool = False):
        now = datetime.now()
        
        while self.calls and (now - self.calls[0]).total_seconds() > self.time_frame:
            self.calls.popleft()
        
        if len(self.calls) >= self.max_calls:
            wait_time = self.time_frame - (now - self.calls[0]).total_seconds()
            wait_time = max(0, wait_time)
            
            if verbose:
                print(f"Đã đạt giới hạn {self.max_calls} request/{self.time_frame}s. Chờ {wait_time:.2f}s...")
            
            time.sleep(wait_time)
            return wait_time
        
        return 0
    
    def add_call(self):
        self.calls.append(datetime.now())

In [3]:
class ApiKeyManager:
    def __init__(self, api_keys, request_limit=1499, verbose=False):
        self.api_keys = api_keys
        self.request_limit = request_limit
        self.request_count = 0
        self.current_key_index = 0
        self.verbose = verbose
    
    def get_current_key(self):
        return self.api_keys[self.current_key_index]
    
    def increment_request_count(self):
        self.request_count += 1
        
        if self.request_count >= self.request_limit:
            self.switch_to_next_key()
    
    def switch_to_next_key(self):
        old_key = self.get_current_key()
        self.current_key_index = (self.current_key_index + 1) % len(self.api_keys)
        self .request_count = 0

        if self.verbose:
            print(f"Đã chuyển đổi từ API key {old_key} sang {self.get_current_key()}")

In [ ]:
def load_merged_chunks(file_path: str) -> List[Dict]:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            chunks = json.load(f)
        
        print(f"Đã tải {len(chunks)} chunks từ file {file_path}")
        return chunks
    
    except Exception as e:
        print(f"Lỗi khi đọc file JSON: {str(e)}")
        return []

In [ ]:
def generate_queries_gemini(
    text: str,
    title: str = "",
    num_questions_per_chunk: int = 1,
    api_key: str = None,
    prompt_template: str = None,
    verbose: bool = False,
    max_retries: int = 3,
    rate_limiter: RateLimiter = None,
    metadata: dict = None,
    api_key_manager: ApiKeyManager = None
) -> Tuple[Dict[str, str], Dict[str, List[str]], Dict[str, str], Dict[str, str]]:

    if api_key_manager:
        current_api_key = api_key_manager.get_current_key()
    elif api_key:
        current_api_key = api_key
    else:
        raise ValueError("API key không được để trống")


    genai.configure(api_key=current_api_key)
    

    model = genai.GenerativeModel('gemini-2.0-flash')
    
    num_questions_per_chunk = 3
    if metadata and 'token_count' in metadata:
        token_count = metadata.get('token_count', 0)
        if token_count > 4000:
            num_questions_per_chunk = 5
            if verbose:
                print(f"Token_count {token_count} > 4000, tạo 5 cặp QA")
        else:
            if verbose:
                print(f"Token_count {token_count} < 4000, tạo 3 cặp QA")


    prompt_template = prompt_template or f"""\
    Dưới đây là một đoạn trích từ quy định của Trường Đại học Sư phạm Kỹ thuật Hưng Yên:
    ----------------------
    Tiêu đề: {{title_str}}
    
    Nội dung:
    {{context_str}}
    ----------------------
    
    Với thông tin đã cho chứ không phải kiến thức có sẵn.
    Bạn là Sinh viên Bậc ĐẠI HỌC, THẠC SĨ, TIẾN SĨ, hãy đặt ra câu hỏi theo nội dung được cung cấp
    
    YÊU CẦU QUAN TRỌNG: Chính xác {num_questions_per_chunk} cặp câu hỏi và đáp án bằng TIẾNG VIỆT.
    
    Quy định:
    1. Mỗi câu hỏi PHẢI bắt đầu bằng "Câu hỏi:"
    2. Mỗi câu trả lời PHẢI bắt đầu bằng "Đáp án:"
    3. KHÔNG được sử dụng ở đầu câu hỏi, và trong câu hỏi bằng "Từ đoạn trích này", "Trong đoạn trích này", "Theo đoạn trích này"
    4. KHÔNG được sử dụng các đại từ "quy định này, văn bản này" mà phải sử dụng tên riêng hoặc từ chỉ rõ.
    5. Bạn PHẢI tạo đúng {num_questions_per_chunk} cặp câu hỏi và đáp án, không nhiều hơn, không ít hơn.
    6. Các câu hỏi nên liên quan đến quy định đào tạo, quy chế học tập, điều kiện tốt nghiệp, v.v.

    VÍ DỤ:
    Câu hỏi: Thời gian đào tạo tối đa cho chương trình đại học hệ chính quy là bao nhiêu năm?
    Câu hỏi: Điều kiện để sinh viên được miễn học, miễn thi các học phần Tiếng Anh là gì?
    Câu hỏi: Quy định về số tín chỉ tối thiểu của chương trình đào tạo đại học là bao nhiêu?
    Câu hỏi: Cách tính điểm GPA cho sinh viên như thế nào?
    
    Format trả lời:
    Câu hỏi: [câu hỏi 1]
    Đáp án: [câu trả lời 1]
    
    Câu hỏi: [câu hỏi 2] 
    Đáp án: [câu trả lời 2]
    
    ...
    """

    queries = {}
    answers = {}
    corpus = {}
    relevant_docs = {}

    for attempt in range(max_retries):
        try:
            query = prompt_template.format(title_str=title, context_str=text)
            

            if rate_limiter:
                rate_limiter.wait_if_needed(verbose)
            

            response = model.generate_content(query)
            

            if api_key_manager:
                api_key_manager.increment_request_count()
                

            if rate_limiter:
                rate_limiter.add_call()

            content = response.text.strip()
            

            questions = re.findall(r'Câu hỏi:(.*?)(?=Đáp án:|$)', content, re.DOTALL)
            answers_found = re.findall(r'Đáp án:(.*?)(?=Câu hỏi:|$)', content, re.DOTALL)
            

            min_pairs = min(len(questions), len(answers_found))
            

            queries.clear()
            answers.clear()
            corpus.clear()
            relevant_docs.clear()
            
            for i in range(min_pairs):
                if i >= num_questions_per_chunk:
                    break
                    
                question_id = str(uuid.uuid4())
                node_id = str(uuid.uuid4())
                queries[question_id] = questions[i].strip()
                answers[question_id] = answers_found[i].strip()
                corpus[node_id] = text
                relevant_docs[question_id] = [node_id]
            
            if len(queries) == num_questions_per_chunk:
                if verbose:
                    print(f"Thành công: Đã tạo đúng {num_questions_per_chunk} câu hỏi")
                break
            elif verbose:
                print(f"Lần thử {attempt+1}: Số câu hỏi tạo ra ({len(queries)}) không khớp với yêu cầu ({num_questions_per_chunk})")
                
            if attempt == max_retries - 1 and verbose:
                print(f"Đã hết số lần thử. Trả về {len(queries)} câu hỏi.")

        except Exception as e:
            if verbose:
                print(f"Lỗi khi tạo câu hỏi (lần thử {attempt+1}): {str(e)}")
            time.sleep(2)
            if attempt == max_retries - 1:
                return {}, {}, {}, {}
    
    return queries, relevant_docs, answers, corpus

In [ ]:
def load_existing_results(output_file):

    try:
        with open(output_file, "r", encoding="utf-8") as f:
            result = json.load(f)
        return result["queries"], result["relevant_docs"], result["answers"], result["corpus"]
    except (FileNotFoundError, json.JSONDecodeError):
        print(f"Không tìm thấy file {output_file} hoặc file không hợp lệ. Tạo mới.")
        return {}, {}, {}, {}

In [ ]:
def save_qa_results(queries, relevant_docs, answers, corpus, output_file):

    result = {
        "queries": queries,
        "relevant_docs": relevant_docs,
        "answers": answers,
        "corpus": corpus
    }
    
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    print(f"Đã lưu kết quả vào file {output_file}")

In [ ]:
def process_merged_chunks_and_generate_qa_gemini(
    json_file_path: str,
    num_questions_per_chunk: int = 3,
    api_key: str = None,
    api_keys: List[str] = None,  
    output_file: str = "utehy_qa_results_gemini.json",
    save_interval: int = 5,
    verbose: bool = True,
    start_index: int = 0,
    end_index: int = None,
    rate_limit: int = 60,  
    request_limit_per_key: int = 1499  
):

    if not api_key and (not api_keys or len(api_keys) == 0):
        raise ValueError("Phải cung cấp ít nhất một API key")
    
    api_key_manager = None
    if api_keys and len(api_keys) > 0:
        api_key_manager = ApiKeyManager(
            api_keys=api_keys,
            request_limit=request_limit_per_key,
            verbose=verbose
        )
    
    if not os.path.exists(json_file_path):
        raise FileNotFoundError(f"Không tìm thấy file: {json_file_path}")
    

    rate_limiter = RateLimiter(max_calls=rate_limit, time_frame=60)
    

    chunks = load_merged_chunks(json_file_path)
    
    if not chunks:
        print(f"Không thể phân tích file JSON: {json_file_path}")
        return {}, {}, {}, {}
    

    if end_index is not None:
        chunks = chunks[start_index:end_index]
    else:
        chunks = chunks[start_index:]

    all_queries, all_relevant_docs, all_answers, all_corpus = load_existing_results(output_file)
    
    chunks_processed = 0
    processed_chunk_ids = set()  
    for i, chunk in enumerate(chunks):
        if verbose:
            print(f"\n=== Đang xử lý chunk {i+1}/{len(chunks)} ===")
            print(f"Tiêu đề: {chunk.get('title', 'Không có tiêu đề')}")
            print(f"Độ dài nội dung: {len(chunk.get('content', ''))} ký tự")
        
        title = chunk.get('title', '')
        content = chunk.get('content', '')
        metadata =chunk.get('metadata', {})

        if len(content.strip()) < 20:
            if verbose:
                print(f"Bỏ qua chunk {i+1} do nội dung quá ngắn")
            continue
        
        chunk_id = str(uuid.uuid5(uuid.NAMESPACE_URL, f"{title}_{len(content)}"))
        
        if any(chunk_id in node_ids for node_ids in all_relevant_docs.values()):
            if verbose:
                print(f"Chunk {i+1} đã được xử lý trước đó. Bỏ qua.")
            processed_chunk_ids.add(chunk_id)
            continue
        
        try
            queries, relevant_docs, answers, corpus = generate_queries_gemini(
                text=content,
                title=title,
                metadata=metadata,
                num_questions_per_chunk=num_questions_per_chunk,
                api_key=api_key if not api_key_manager else None,  
                api_key_manager=api_key_manager,
                verbose=verbose,
                rate_limiter=rate_limiter
            )
            
            new_queries = {}
            for question_id, question in queries.items():
                short_title = title.split('\n')[0] if '\n' in title else title
                short_title = short_title[:50] + '...' if len(short_title) > 50 else short_title
                new_query = f"[{short_title}] {question}"
                new_queries[question_id] = new_query
            
            new_corpus = {}
            new_relevant_docs = {}
            for qid, q in new_queries.items():
                if qid in relevant_docs:
                    original_node_ids = relevant_docs[qid]
                    new_node_ids = []
                    
                    for original_id in original_node_ids:
                        if original_id in corpus:
                            new_corpus[chunk_id] = {
                                "content": corpus[original_id],
                                "title": title
                            }
                            new_node_ids.append(chunk_id)
                    
                    if new_node_ids:
                        new_relevant_docs[qid] = new_node_ids
            

            all_queries.update(new_queries)
            all_relevant_docs.update(new_relevant_docs)
            all_answers.update(answers)
            all_corpus.update(new_corpus)
            
            processed_chunk_ids.add(chunk_id)
            chunks_processed += 1
            
            if verbose:
                print(f"Đã tạo {len(queries)} câu hỏi từ chunk này")
                for qid, q in queries.items():
                    print(f"Q: {q}")
                    print(f"A: {answers[qid]}")
                    print("-" * 40)
            

            if chunks_processed % save_interval == 0:
                save_qa_results(all_queries, all_relevant_docs, all_answers, all_corpus, output_file)
                print(f"Đã lưu tiến trình sau khi xử lý {chunks_processed} chunk")
                

        
        except Exception as e:
            if verbose:
                print(f"Lỗi khi xử lý chunk {i+1}: {str(e)}")

            save_qa_results(all_queries, all_relevant_docs, all_answers, all_corpus, output_file)
            print(f"Đã lưu tiến trình trước khi gặp lỗi.")

    save_qa_results(all_queries, all_relevant_docs, all_answers, all_corpus, output_file)
    
    if verbose:
        print(f"\nHoàn thành: {len(processed_chunk_ids)} chunk đã được xử lý thành công.")
        print(f"Tổng cộng: {len(all_queries)} câu hỏi được tạo.")
    
    return all_queries, all_relevant_docs, all_answers, all_corpus

In [ ]:
def main_gemini():

    api_keys = [
           
    ]

    json_file_path = "/Users/toan/Working/UTEHY_CHATBOT/data_process/all_chunks_merged_newest.json"
    num_questions_per_chunk = 3 
    output_file = "/Users/toan/Working/UTEHY_CHATBOT/data_gen/utehy_qa_gemini_all_1k.json"  
    save_interval = 1 
    
    process_merged_chunks_and_generate_qa_gemini(
        json_file_path=json_file_path,
        num_questions_per_chunk=num_questions_per_chunk,
        api_keys=api_keys,
        output_file=output_file,
        save_interval=save_interval,
        verbose=True,
        start_index=0, 
        end_index=None, 
        rate_limit=14,    
        request_limit_per_key=1499 
    )

if __name__ == "__main__":
    main_gemini()  # Sử dụng Gemini


Đã tải 278 chunks từ file /Users/toan/Working/UTEHY_CHATBOT/data_process/all_chunks_merged_newest.json
Không tìm thấy file /Users/toan/Working/UTEHY_CHATBOT/data_gen/utehy_qa_gemini_all_1k.json hoặc file không hợp lệ. Tạo mới.

=== Đang xử lý chunk 1/278 ===
Tiêu đề: # BỘ GIÁO DỤC VÀ ĐÀO TẠO
TRƯỜNG ĐẠI HỌC SP KỸ THUẬT HƯNG YÊN
Số: **818**/QĐ-ĐHSPKT
Hưng Yên, ngày **26** tháng 5 năm 2021

## QUYẾT ĐỊNH

Về việc ban hành Quy chế bảo quản, lưu giữ, sử dụng, cấp phát văn bằng,
chứng chỉ, chứng nhận của Trường Đại học Sư phạm Kỹ thuật Hưng Yên
Độ dài nội dung: 1461 ký tự
Token_count 779 < 4000, tạo 3 cặp QA
Thành công: Đã tạo đúng 3 câu hỏi
Đã tạo 3 câu hỏi từ chunk này
Q: Quyết định số 818/QĐ-ĐHSPKT của Trường Đại học Sư phạm Kỹ thuật Hưng Yên ban hành ngày 26 tháng 5 năm 2021 thay thế cho quyết định nào?
A: Quyết định số 818/QĐ-ĐHSPKT của Trường Đại học Sư phạm Kỹ thuật Hưng Yên ban hành ngày 26 tháng 5 năm 2021 thay thế cho Quyết định số 1923/QĐ-ĐHSPKT ngày 26/9/2019 của Hiệu trưởng trườ